In [1]:
import pandas as pd
import os
import numpy as np
import datetime
print(datetime.datetime.now())

# Revised based on the 1st version new run, remove the 8+22 together

stores_add_back_list=[1,410,1379,1650,4648,5272,5324]
stores_add_back_list=[str(x) for x in stores_add_back_list]
# Based on the filetered excel in the email from Kayla "RE: Big Lots - reoccurring quad 3 stores" on 2019-09-26 4:55 pm 

os.getcwd()

2019-09-27 11:36:12.784360


'/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/add_and_remove_stores'

In [2]:
file_closing_stores=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/Additional Stores to Remove_Q3 9.11.xlsx")
file_closing_stores.sheet_names

['Closures & Non Movers #1 ',
 'Additional Stores to Remove #2',
 'Additional Stores to Remove (2)',
 'Full Store List',
 'SOTF List',
 'Removed quad 3 Stores already']

In [3]:
df_closing_22_stores=file_closing_stores.parse("Closures & Non Movers #1 ",dtype=str)
list_closing_22_stores=df_closing_22_stores['Store ID'].unique().tolist()
print(len(list_closing_22_stores))

df_closing_159_stores=file_closing_stores.parse("Additional Stores to Remove #2",dtype=str)
list_closing_159_stores=df_closing_159_stores['Store ID'].unique().tolist()
print(len(list_closing_159_stores))

list_closing_8_stores=['182','1095','1499','4061','4070','4125','4441','4529']

total_closing_stores=list(set(list_closing_8_stores+list_closing_159_stores+list_closing_22_stores))
print("total_closing_stores: ",len(total_closing_stores))
total_closing_stores=[x for x in total_closing_stores if x not in stores_add_back_list]
print("total_closing_stores added back: ",len(total_closing_stores))

22
159
total_closing_stores:  188
total_closing_stores added back:  181


In [5]:
output_20190718=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
output_20190718.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [6]:
df_old_by_store=output_20190718.parse("view_by_store",dtype=str)
df_old_by_store['TA_num']=df_old_by_store['TA_num'].astype(int)
df_old_by_store_closing=df_old_by_store[df_old_by_store['location_id'].isin(total_closing_stores)]

In [7]:
df_old_by_store_closing['trans_P_zips_70_within_TA']=df_old_by_store_closing['trans_P_zips_70_within_TA'].apply(eval)
df_old_by_store_closing['trans_S_zips_70_within_TA']=df_old_by_store_closing['trans_S_zips_70_within_TA'].apply(eval)

list_zips_for_closing_stores=df_old_by_store_closing['trans_P_zips_70_within_TA'].sum()+df_old_by_store_closing['trans_S_zips_70_within_TA'].sum()
list_zips_for_closing_stores=list(set(list_zips_for_closing_stores))

df_new_by_store=df_old_by_store[~df_old_by_store['location_id'].isin(total_closing_stores)]
df_new_by_store=df_new_by_store.sort_values("TA_num")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
df_new_by_store['zips_in_10']=df_new_by_store['zips_in_10'].apply(eval)
df_new_by_store['trans_P_zips_70_within_TA']=df_new_by_store['trans_P_zips_70_within_TA'].apply(eval)
df_new_by_store['trans_S_zips_70_within_TA']=df_new_by_store['trans_S_zips_70_within_TA'].apply(eval)

df_new_by_store.shape

(1227, 17)

In [9]:
df_new_by_TA_stores=df_new_by_store.groupby(["TA_num",'ta_name'])['location_id'].apply(list).to_frame().reset_index().rename(columns={"location_id":"store_list"})
df_new_by_TA_P_S_10_zips=df_new_by_store.groupby(["TA_num",'ta_name'])['trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10'].sum().reset_index()
df_new_by_TA_P_S_10_zips=df_new_by_TA_P_S_10_zips.rename(columns={"trans_P_zips_70_within_TA":"trans_P_zips","trans_S_zips_70_within_TA":"trans_S_zips","zips_in_10":"distance_10_zips"})
df_new_by_TA=pd.merge(df_new_by_TA_stores,df_new_by_TA_P_S_10_zips,on=["TA_num",'ta_name'],how="outer")
print(df_new_by_TA.shape)

for col in ['store_list','trans_P_zips','trans_S_zips','distance_10_zips']:
    df_new_by_TA[col]=df_new_by_TA[col].apply(lambda x: list(set(x)))
df_new_by_TA['store_count']=df_new_by_TA['store_list'].apply(len)
df_new_by_TA=df_new_by_TA.sort_values("TA_num")
df_new_by_TA.head(2)

(627, 6)


,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,distance_10_zips,store_count
0,1,multiple_1,"[1255, 4589, 1462, 1949, 1878, 1650, 4143, 413...","[76208, 75043, 75407, 76092, 75090, 76210, 751...","[76053, 75424, 76208, 76092, 75090, 76210, 734...",[],20
1,2,multiple_2,"[5220, 1379, 1079, 1449, 1668, 5092, 5356, 537...","[43318, 43040, 43065, 43123, 43078, 44906, 437...","[43347, 43011, 43050, 43235, 43822, 43773, 430...",[],16


In [10]:
df_old_by_TA=output_20190718.parse("view_by_TA",dtype=str)
print(df_old_by_TA.shape)
df_old_by_TA.head(2)

(706, 7)


,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,distance_10_zips,store_count
0,1,multiple_1,"['1949', '1255', '4543', '1027', '1072', '1106...","['75002', '75006', '75007', '75020', '75023', ...","['73448', '73459', '74701', '74733', '75001', ...",nan,21
1,2,multiple_2,"['5220', '5356', '1666', '1667', '1668', '77',...","['43015', '43016', '43017', '43021', '43026', ...","['43003', '43004', '43008', '43009', '43011', ...",nan,17


In [11]:
df_old_unique_zips_full_footprint=output_20190718.parse("unique_zips_full_footprint",dtype=str)
print(df_old_unique_zips_full_footprint.shape)
df_old_unique_zips_full_footprint['zip_type'].unique()

(12980, 2)


array(['trans_P', 'trans_S', 'zips_10'], dtype=object)

In [12]:
df_new_unique_zips_full_footprint=pd.DataFrame(columns={"zip_type",'zip_cd'})
#
all_P_zips=list(set(df_new_by_TA['trans_P_zips'].sum()))
all_S_zips=list(set(df_new_by_TA['trans_S_zips'].sum()))
all_10_zips=list(set(df_new_by_TA['distance_10_zips'].sum()))
#
all_S_zips=[x for x in all_S_zips if x not in all_P_zips]
#
all_10_zips=[x for x in all_10_zips if x not in all_P_zips]
all_10_zips=[x for x in all_10_zips if x not in all_S_zips]


df_P_zips=pd.DataFrame({"zip_type":["trans_P"]*len(all_P_zips),"zip_cd":all_P_zips},index=[x for x in range(len(all_P_zips))])

df_S_zips=pd.DataFrame({"zip_type":["trans_S"]*len(all_S_zips),"zip_cd":all_S_zips},index=[x for x in range(len(all_S_zips))])

df_10_zips=pd.DataFrame({"zip_type":["zips_10"]*len(all_10_zips),"zip_cd":all_10_zips},index=[x for x in range(len(all_10_zips))])
df_new_unique_zips_full_footprint=pd.concat([df_P_zips,df_S_zips,df_10_zips])
print(df_new_unique_zips_full_footprint.shape)

(11878, 2)


In [13]:
df_new_unique_zips_full_footprint=df_new_unique_zips_full_footprint[['zip_type','zip_cd']]

In [14]:
writer=pd.ExcelWriter("./BL_final_TA_updated_remove_8_22_159_stores_added_back_7_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_new_unique_zips_full_footprint.to_excel(writer,"unique_zips_full_footprint",index=False)
df_new_by_TA.to_excel(writer,"view_by_TA",index=False)
df_new_by_store.to_excel(writer,"view_by_store",index=False)
df_store_removed=pd.DataFrame({"removed_stores":total_closing_stores},index=[x for x in range(len(total_closing_stores))])
df_store_removed.to_excel(writer,"stores_removed",index=False)
writer.save()